In [ ]:
from google.colab import files
files.upload()

In [2]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d mathurinache/ethereum-tweets

  0% 0.00/1.35M [00:00<?, ?B/s]
100% 1.35M/1.35M [00:00<00:00, 91.6MB/s]


In [7]:
import zipfile

unzip = zipfile.ZipFile("ethereum-tweets.zip","r")
unzip.extractall()
unzip.close()

In [103]:
import pandas as pd

data = pd.read_csv("Ethereum_tweets.csv")

In [104]:
data.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,#1 Crypto Currency TRADING,NaN,"Profit thought trading, Learn , Trade & Earn💲....",2021-11-10 10:42:01+00:00,4,0,0,False,2021-11-15 07:26:39+00:00,Get upto 1000 that’s 10x with our premium spot...,NaN,Twitter for Android,False
1,Reyrey,Kuala Lumpur,Artist | NFT Broker & Collector for AnideaNFT ...,2011-03-07 07:22:57+00:00,343,650,589,False,2021-11-15 07:26:35+00:00,"Adopted a Hypocat #053, “Crazy Rich Cat” 🤑💵💵💵🤩...","['SupportingCreators', 'CryptoArt', 'CryptoArt...",Twitter for iPhone,False
2,akter jahan,NaN,NaN,2021-10-28 13:28:05+00:00,1,2,0,False,2021-11-15 07:26:32+00:00,Top 10 Coins by Social Engagement over the las...,"['LunarCrush', 'bitcoin', 'dogecoin']",Twitter Web App,False
3,riyan,NaN,i always paricipated your project,2021-09-16 16:59:11+00:00,11,127,99,False,2021-11-15 07:26:30+00:00,Requesting faucet funds into 0x5278942b39deD8c...,"['Rinkeby', 'Ethereum']",Twitter Web App,False
4,Michelle Eriksen,NaN,"Art Creator, NFT, Crypto",2021-08-11 06:40:08+00:00,522,1428,16,False,2021-11-15 07:26:30+00:00,New Listing BTS 0.008ETH !! \n#NFTs #nftcolle...,"['NFTs', 'nftcollector', 'NFTCommunity', 'art'...",Twitter Web App,False


In [105]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_name         10000 non-null  object
 1   user_location     4961 non-null   object
 2   user_description  8628 non-null   object
 3   user_created      10000 non-null  object
 4   user_followers    10000 non-null  int64 
 5   user_friends      10000 non-null  int64 
 6   user_favourites   10000 non-null  int64 
 7   user_verified     10000 non-null  bool  
 8   date              10000 non-null  object
 9   text              10000 non-null  object
 10  hashtags          7528 non-null   object
 11  source            10000 non-null  object
 12  is_retweet        10000 non-null  bool  
dtypes: bool(2), int64(3), object(8)
memory usage: 879.0+ KB


In [106]:
data.isnull().sum()

user_name              0
user_location       5039
user_description    1372
user_created           0
user_followers         0
user_friends           0
user_favourites        0
user_verified          0
date                   0
text                   0
hashtags            2472
source                 0
is_retweet             0
dtype: int64

In [107]:
data = data.drop(data[["user_location","user_description","hashtags","user_created","user_followers","user_friends","user_favourites","user_verified","source","is_retweet"]], axis= 1)

In [108]:
data.isnull().sum()

user_name    0
date         0
text         0
dtype: int64

In [109]:
data.head()

,user_name,date,text
0,#1 Crypto Currency TRADING,2021-11-15 07:26:39+00:00,Get upto 1000 that’s 10x with our premium spot...
1,Reyrey,2021-11-15 07:26:35+00:00,"Adopted a Hypocat #053, “Crazy Rich Cat” 🤑💵💵💵🤩..."
2,akter jahan,2021-11-15 07:26:32+00:00,Top 10 Coins by Social Engagement over the las...
3,riyan,2021-11-15 07:26:30+00:00,Requesting faucet funds into 0x5278942b39deD8c...
4,Michelle Eriksen,2021-11-15 07:26:30+00:00,New Listing BTS 0.008ETH !! \n#NFTs #nftcolle...


In [110]:
data["user_name"].value_counts()

REAL-TIME STOCK NEWS ALERT       174
Stock Crypto News (Real-Time)    171
SorareBot by SorareData           99
frieslips.nft                     85
Alireza Mirhadi                   83
                                ... 
Tony                               1
molisehao                          1
md_sufee                           1
Mosaic Shaman 🍄                    1
doe                                1
Name: user_name, Length: 4932, dtype: int64

In [111]:
data["text"]

0       Get upto 1000 that’s 10x with our premium spot...
1       Adopted a Hypocat #053, “Crazy Rich Cat” 🤑💵💵💵🤩...
2       Top 10 Coins by Social Engagement over the las...
3       Requesting faucet funds into 0x5278942b39deD8c...
4       New Listing BTS 0.008ETH !! \n#NFTs  #nftcolle...
                              ...                        
9995    Pepe the gunman II\n\nhttps://t.co/xTOJSvLwX9 ...
9996    @missteennfts 🔥 TOODLER 1:1 EDITION 🔥\nPlease ...
9997    Hello guys! mind to view my project at opensea...
9998    Ethereum: One major misconception that will no...
9999    Giovanni Simeone bought for 0.238 ETH (1100.25...
Name: text, Length: 10000, dtype: object

##**Processing the Data**

In [112]:
import re
from gensim.parsing.preprocessing import remove_stopwords
import string
import unicodedata

def lower_case(text):
    return text.lower()

def remove_punctuation(text):
    text_list = [char for char in text if char not in string.punctuation]
    clean_text = "".join(text_list)
    return clean_text

def remove_numbers(text):
    text_list = []
    for i in text:
        if not re.search("\d", i):
            text_list.append(i)
    return "".join(text_list)

def remove_special_char(text):
    pat =  r'[^a-zA-z0-9]' 
    return re.sub(pat, " ", text)

def remove_square_bracket(text):
    clean_text = re.sub(r'\[[0-9]*\]', ' ', text)
    return clean_text

def remove_white_spaces_char(text):
    clean_text = re.sub(r'\s+', ' ', text)
    
    return clean_text

def remove_unicode_char(text):
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text


def remove_user(text):
    clean_text = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', text)
    
    return clean_text

def remove_links(text):
    clean_text = re.sub(r'http\S+', '', text)
    clean_text = re.sub(r'bit.ly/\S+', '', text)
    clean_text = clean_text.strip('[link]')
    
    return clean_text

def remove_html(text):
    html = re.compile('<.*?>')
    
    return html.sub(r'',text)

def remove_email(text):
    email = re.compile(r'[\w\.-]+@[\w\.-]+')
    return email.sub(r'',text)

def remove_stopwords_from_text(text):
    
    clean_text = remove_stopwords(text)
    
    return clean_text


In [113]:
data["clean_text"] = data["text"].apply(lower_case)
data["clean_text"] = data["clean_text"].apply(remove_punctuation)
data["clean_text"] = data["clean_text"].apply(remove_numbers)
data["clean_text"] = data["clean_text"].apply(remove_special_char)
data["clean_text"] = data["clean_text"].apply(remove_square_bracket)
data["clean_text"] = data["clean_text"].apply(remove_white_spaces_char)
data["clean_text"] = data["clean_text"].apply(remove_unicode_char)
data["clean_text"] = data["clean_text"].apply(remove_user)
data["clean_text"] = data["clean_text"].apply(remove_links)
data["clean_text"] = data["clean_text"].apply(remove_html)
data["clean_text"] = data["clean_text"].apply(remove_email)
data["clean_text"] = data["clean_text"].apply(remove_stopwords_from_text)

In [114]:
data.head()

,user_name,date,text,clean_text
0,#1 Crypto Currency TRADING,2021-11-15 07:26:39+00:00,Get upto 1000 that’s 10x with our premium spot...,upto s x premium spot trading signals accurate...
1,Reyrey,2021-11-15 07:26:35+00:00,"Adopted a Hypocat #053, “Crazy Rich Cat” 🤑💵💵💵🤩...",adopted hypocat crazy rich cat hypocatclub meo...
2,akter jahan,2021-11-15 07:26:32+00:00,Top 10 Coins by Social Engagement over the las...,coins social engagement hrs lunarcrush btc bit...
3,riyan,2021-11-15 07:26:30+00:00,Requesting faucet funds into 0x5278942b39deD8c...,requesting faucet funds xbdedcbdadeaecefdca ri...
4,Michelle Eriksen,2021-11-15 07:26:30+00:00,New Listing BTS 0.008ETH !! \n#NFTs #nftcolle...,ew listing bts eth nfts nftcollector nftcommun...


In [115]:
data = data.drop(data[["user_name","date"]], axis = 1)

In [116]:
data.head()

,text,clean_text
0,Get upto 1000 that’s 10x with our premium spot...,upto s x premium spot trading signals accurate...
1,"Adopted a Hypocat #053, “Crazy Rich Cat” 🤑💵💵💵🤩...",adopted hypocat crazy rich cat hypocatclub meo...
2,Top 10 Coins by Social Engagement over the las...,coins social engagement hrs lunarcrush btc bit...
3,Requesting faucet funds into 0x5278942b39deD8c...,requesting faucet funds xbdedcbdadeaecefdca ri...
4,New Listing BTS 0.008ETH !! \n#NFTs #nftcolle...,ew listing bts eth nfts nftcollector nftcommun...


##**Tweets Polarity**

In [117]:
from textblob import TextBlob

def finding_polarity(text):
    clean_text = TextBlob(str(text))
    pol = clean_text.sentiment.polarity
    if(pol==0):
        return "Neutral"
    elif(pol>0 and pol<=0.3):
        return "Weakly Positive"
    elif(pol>0.3 and pol<=0.6):
        return "Positive"
    elif(pol>0.6 and pol<=1):
        return "Strongly Positive"
    elif(pol>-0.3 and pol<=0):
        return "Weakly Negative"
    elif(pol>-0.6 and pol<=-0.3):
        return "Negative"
    elif(pol>-1 and pol<=-0.6):
        return "Strongly Negative"

In [118]:
data["polarity"] = data["clean_text"].apply(finding_polarity)

In [119]:
data.head()

,text,clean_text,polarity
0,Get upto 1000 that’s 10x with our premium spot...,upto s x premium spot trading signals accurate...,Weakly Positive
1,"Adopted a Hypocat #053, “Crazy Rich Cat” 🤑💵💵💵🤩...",adopted hypocat crazy rich cat hypocatclub meo...,Weakly Negative
2,Top 10 Coins by Social Engagement over the las...,coins social engagement hrs lunarcrush btc bit...,Weakly Positive
3,Requesting faucet funds into 0x5278942b39deD8c...,requesting faucet funds xbdedcbdadeaecefdca ri...,Neutral
4,New Listing BTS 0.008ETH !! \n#NFTs #nftcolle...,ew listing bts eth nfts nftcollector nftcommun...,Neutral


##**Labeling the Tweets**

In [120]:
# categorizing the text

from textblob import TextBlob

def positive_negative(text):
  clean_text = TextBlob(str(text))
  pol = clean_text.sentiment.polarity

  

  if(pol==0):
    return 1
  elif(pol>0 and pol<=0.3):
    return 1
  elif(pol>0.3 and pol<=0.6):
    return 1
  elif(pol>0.6 and pol<=1):
    return 1
  elif(pol>-0.3 and pol<=0):
    return 0
  elif(pol>-0.6 and pol<=-0.3):
    return 0
  elif(pol>-1 and pol<=-0.6):
    return 0


In [121]:

data["label"] = data["clean_text"].apply(positive_negative)

In [122]:
data.head()

,text,clean_text,polarity,label
0,Get upto 1000 that’s 10x with our premium spot...,upto s x premium spot trading signals accurate...,Weakly Positive,1.0
1,"Adopted a Hypocat #053, “Crazy Rich Cat” 🤑💵💵💵🤩...",adopted hypocat crazy rich cat hypocatclub meo...,Weakly Negative,0.0
2,Top 10 Coins by Social Engagement over the las...,coins social engagement hrs lunarcrush btc bit...,Weakly Positive,1.0
3,Requesting faucet funds into 0x5278942b39deD8c...,requesting faucet funds xbdedcbdadeaecefdca ri...,Neutral,1.0
4,New Listing BTS 0.008ETH !! \n#NFTs #nftcolle...,ew listing bts eth nfts nftcollector nftcommun...,Neutral,1.0


In [102]:
data["label"].value_counts()

1.0    8282
0.0    1640
Name: label, dtype: int64

In [132]:
length = sum([len(word.split()) for word in data["clean_text"]])

In [133]:
length

176391

In [151]:
# text vectorizing

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features= 3500)

x = vectorizer.fit_transform(data["clean_text"]).toarray()

y = data["label"].values.astype("int")

In [152]:
x.shape, y.shape

((10000, 3500), (10000,))

In [153]:
# splitting the data

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.25, random_state= 42)

In [154]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((7500, 3500), (2500, 3500), (7500,), (2500,))

##**Naive Bayes Classifier**

In [161]:
from sklearn.naive_bayes import GaussianNB

nav_model = GaussianNB()

nav_model.fit(x_train,y_train)



GaussianNB()

In [162]:
print(f"Training Accuracy : {nav_model.score(x_train,y_train) * 100 : 0.2f} %")
print(f"Test Accuracy : {nav_model.score(x_test,y_test)* 100 : 0.2f} %")

Training Accuracy :  85.25 %
Test Accuracy :  80.68 %


##**Support Vector Machine**

In [163]:
from sklearn.svm import SVC

svc_model = SVC()

svc_model.fit(x_test,y_test)



SVC()

In [164]:
print(f"Training Accuracy: {svc_model.score(x_train, y_train)*100 : 0.2f} %")
print(f"Test Accuracy: {svc_model.score(x_test,y_test) * 100 : 0.2f} %")

Training Accuracy:  94.33 %
Test Accuracy:  97.16 %
